# Azure ML v2 SDK — redeploy LLM (Azure OpenAI) under existing Managed Online Endpoint

Cel: podmienić **jeden deployment** pod istniejącym endpointem, zrobić mu **100% traffic** (konfigurowalne, bez hardcode), a po drodze mieć **deterministyczne testy diagnostyczne**: czy działa środowisko, czy SDK, czy uprawnienia, czy build obrazu, czy `score.py`, czy ruch/keys.

Ten notebook zakłada:
- Endpoint już istnieje.
- Pod endpointem są 2+ deploymenty.
- Chcesz stworzyć/odświeżyć jeden deployment (np. `llm`), przetestować go, a następnie (opcjonalnie) przestawić traffic.

Źródła referencyjne: dokumentacja Azure ML online endpoints i traffic allocation. citeturn0search1turn0search3turn0search5


In [ ]:
# 0) (Opcjonalnie) aktualizacja paczek w notebook compute
# Uwaga: jeśli działasz na Compute Instance w AML, %pip jest OK. Lokalnie użyj pip/conda.
%pip install -U azure-ai-ml azure-identity azure-core pyyaml pandas openpyxl python-dotenv openai requests


## 1) Konfiguracja — w jednym miejscu

Ważne:
- **Nie** hardcodujemy trafficu w kodzie „na zawsze”. Używamy zmiennych sterujących i funkcji `set_traffic(...)`.
- `NEW_DEPLOYMENT_NAME` to deployment, który podmieniasz/odświeżasz.


In [ ]:
import os
from pathlib import Path

# === AML Workspace ===
SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID", "REPLACE_ME")
RESOURCE_GROUP  = os.getenv("AZURE_RESOURCE_GROUP",  "REPLACE_ME")
WORKSPACE_NAME  = os.getenv("AZUREML_WORKSPACE",     "REPLACE_ME")

# === Existing Online Endpoint ===
ENDPOINT_NAME = os.getenv("AML_ENDPOINT_NAME", "REPLACE_ME")  # istniejący endpoint

# === Deployment to (re)create ===
NEW_DEPLOYMENT_NAME = os.getenv("AML_NEW_DEPLOYMENT_NAME", "llm")  # ten podmieniasz

# Compute for deployment
INSTANCE_TYPE  = os.getenv("AML_INSTANCE_TYPE", "Standard_DS3_v2")
INSTANCE_COUNT = int(os.getenv("AML_INSTANCE_COUNT", "1"))

# Folder z kodem scoringu
CODE_DIR = Path("./src_llm")
CODE_DIR.mkdir(exist_ok=True, parents=True)

# Environment asset (z conda yaml)
CONDA_YAML_PATH = Path("./conda_env.yml")  # w tym środowisku notebooka masz już plik z uploadu; tu trzymamy kopię
ENV_NAME = os.getenv("AML_ENV_NAME", "openai-rag-demand-labeler")
ENV_VERSION = os.getenv("AML_ENV_VERSION", "1")  # możesz bumpować gdy zmieniasz zależności

# === Traffic control ===
SET_TRAFFIC_AFTER_SMOKE_TEST = os.getenv("AML_SET_TRAFFIC", "true").lower() == "true"
NEW_DEPLOYMENT_TRAFFIC_PERCENT = int(os.getenv("AML_NEW_DEPLOYMENT_TRAFFIC_PERCENT", "100"))

assert 0 <= NEW_DEPLOYMENT_TRAFFIC_PERCENT <= 100, "Traffic percent must be 0..100"
print("Config OK")


## 2) Diagnostyka środowiska notebooka (lokalnego/Compute Instance)

To ma odpowiedzieć na pytania:
- czy paczki są zainstalowane?
- czy wersje są sensowne?
- czy widzisz `conda_env.yml`?


In [ ]:
import sys, platform, importlib, pkgutil
import azure.ai.ml
import azure.identity
import openai

print("Python:", sys.version)
print("Platform:", platform.platform())
print("azure-ai-ml:", azure.ai.ml.__version__)
print("azure-identity:", azure.identity.__version__)
print("openai:", openai.__version__)
print("CONDA_YAML_PATH exists?:", CONDA_YAML_PATH.exists(), str(CONDA_YAML_PATH.resolve()))


## 3) Wgraj/odśwież `conda_env.yml` w katalogu roboczym notebooka

Masz załączony plik `/mnt/data/conda_env.yml`. Skopiujemy go do bieżącego katalogu, aby Environment asset mógł się z niego budować.


In [ ]:
import shutil

SRC_UPLOADED_CONDA = Path("/mnt/data/conda_env.yml")
if SRC_UPLOADED_CONDA.exists():
    shutil.copy2(SRC_UPLOADED_CONDA, CONDA_YAML_PATH)
    print("Copied conda yaml from:", SRC_UPLOADED_CONDA)
else:
    raise FileNotFoundError(f"Nie widzę {SRC_UPLOADED_CONDA}. Dodaj plik albo zmień ścieżkę.")

print(CONDA_YAML_PATH.read_text(encoding="utf-8")[:1000])


## 4) Wygeneruj kod inferencji (`score.py`) i przykładowe dane `demands.xlsx`

Ten krok jest **samowystarczalny**: w folderze `CODE_DIR` powstanie `score.py` i `demands.xlsx`.


In [ ]:
import pandas as pd

# --- demands.xlsx (przykład; możesz podmienić na swoje) ---
demands_path = CODE_DIR / "demands.xlsx"
if not demands_path.exists():
    df = pd.DataFrame([
        {"demand_id":"motor-earthing","demand":"motor - earthing","description":"Wymóg podłączenia silnika/ramy do PE / uziemienia."},
        {"demand_id":"motor-ip","demand":"motor - IP","description":"Wymóg klasy szczelności (np. IP55, IP66)."},
        {"demand_id":"motor-iso9001","demand":"motor - ISO9001","description":"Wymóg certyfikacji ISO 9001 (dokument/certyfikat)."},
    ])
    df.to_excel(demands_path, index=False)
    print("Created:", demands_path)
else:
    print("Exists:", demands_path)

# --- score.py ---
score_path = CODE_DIR / "score.py"
score_path.write_text('import os\nimport json\nimport logging\nimport time\nfrom typing import Any, Dict, List, Optional\n\nimport numpy as np\nimport pandas as pd\nfrom openai import AzureOpenAI\n\nlogging.basicConfig(level=logging.INFO)\nlogger = logging.getLogger(__name__)\n\nclient: Optional[AzureOpenAI] = None\nDEMANDS: List[Dict[str, str]] = []\nDEMAND_EMB: Optional[np.ndarray] = None\n\nAZURE_CHAT_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT", "").strip()\nAZURE_EMBED_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBED_DEPLOYMENT", "").strip()\n\nTOP_K_RAG = int(os.getenv("TOP_K_RAG", "8"))\nMAX_LABELS_PER_CHUNK = int(os.getenv("MAX_LABELS_PER_CHUNK", "3"))\nMIN_KEEP_PROBA = float(os.getenv("MIN_KEEP_PROBA", "0.30"))\nMAX_TEXT_CHARS = int(os.getenv("MAX_TEXT_CHARS", "4000"))\n\ndef _l2_normalize(x: np.ndarray) -> np.ndarray:\n    denom = (np.linalg.norm(x, axis=1, keepdims=True) + 1e-12)\n    return x / denom\n\ndef _safe_float(v: Any, default: float = 0.0) -> float:\n    try:\n        return float(v)\n    except Exception:\n        return default\n\ndef _safe_json_loads(s: str) -> Optional[dict]:\n    try:\n        return json.loads(s)\n    except Exception:\n        return None\n\ndef init():\n    global client, DEMANDS, DEMAND_EMB, AZURE_CHAT_DEPLOYMENT, AZURE_EMBED_DEPLOYMENT\n    logger.info("INIT: starting...")\n\n    endpoint = (os.getenv("AZURE_OPENAI_ENDPOINT") or "").strip()\n    api_key = (os.getenv("AZURE_OPENAI_API_KEY") or "").strip()\n    api_version = (os.getenv("AZURE_OPENAI_API_VERSION") or "2024-12-01-preview").strip()\n\n    AZURE_CHAT_DEPLOYMENT = (os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT") or "").strip()\n    AZURE_EMBED_DEPLOYMENT = (os.getenv("AZURE_OPENAI_EMBED_DEPLOYMENT") or "").strip()\n\n    if not endpoint or not api_key:\n        logger.error("Missing AZURE_OPENAI_ENDPOINT or AZURE_OPENAI_API_KEY.")\n        return\n    if not AZURE_CHAT_DEPLOYMENT or not AZURE_EMBED_DEPLOYMENT:\n        logger.error("Missing AZURE_OPENAI_CHAT_DEPLOYMENT or AZURE_OPENAI_EMBED_DEPLOYMENT.")\n        return\n\n    client = AzureOpenAI(azure_endpoint=endpoint, api_key=api_key, api_version=api_version)\n\n    base_dir = os.path.dirname(os.path.abspath(__file__))\n    demands_path = os.path.join(base_dir, "demands.xlsx")\n    if not os.path.exists(demands_path):\n        logger.error(f"demands.xlsx not found at: {demands_path}")\n        return\n\n    df = pd.read_excel(demands_path)\n    required_cols = {"demand_id", "demand", "description"}\n    if not required_cols.issubset(df.columns):\n        logger.error(f"demands.xlsx missing columns. Required: {required_cols}, got: {set(df.columns)}")\n        return\n\n    DEMANDS = []\n    embed_inputs: List[str] = []\n    for _, row in df.iterrows():\n        did = str(row["demand_id"]).strip()\n        name = str(row["demand"]).strip()\n        desc = str(row["description"]).strip()\n        if not did or did.lower() == "nan":\n            continue\n        if not name or name.lower() == "nan":\n            continue\n        DEMANDS.append({"id": did, "name": name, "description": desc})\n        embed_inputs.append(f"Name: {name}\\nClarification: {desc}")\n\n    if not DEMANDS:\n        logger.error("No demands loaded from Excel (DEMANDS is empty).")\n        return\n\n    t0 = time.time()\n    emb = client.embeddings.create(model=AZURE_EMBED_DEPLOYMENT, input=embed_inputs)\n    DEMAND_EMB = _l2_normalize(np.array([e.embedding for e in emb.data], dtype=np.float32))\n    logger.info(f"INIT: loaded {len(DEMANDS)} demands. Embedding time: {time.time() - t0:.2f}s")\n\ndef _retrieve_candidates(chunk_text: str, k: int) -> List[Dict[str, str]]:\n    if client is None or DEMAND_EMB is None or not DEMANDS:\n        return []\n    emb = client.embeddings.create(model=AZURE_EMBED_DEPLOYMENT, input=[chunk_text])\n    q = _l2_normalize(np.array([emb.data[0].embedding], dtype=np.float32))\n    sims = DEMAND_EMB @ q[0]\n    idx = np.argsort(-sims)[:k]\n    return [DEMANDS[i] for i in idx]\n\ndef _llm_classify_chunk(chunk_id: str, chunk_text: str, candidates: List[Dict[str, str]]) -> Dict[str, Any]:\n    if client is None:\n        return {"chunkId": chunk_id, "demandIds": [], "explanation": "AzureOpenAI client not initialized."}\n    if not candidates:\n        return {"chunkId": chunk_id, "demandIds": [], "explanation": "No candidates from retrieval."}\n\n    demands_context = "\\n".join(\n        [f"- id: {d[\'id\']}\\n  name: {d[\'name\']}\\n  clarification: {d[\'description\']}" for d in candidates]\n    )\n\n    system = (\n        "You label customer requirements in product documentation. "\n        "You MUST only choose from the provided demands. "\n        "Match ONLY when the chunk clearly satisfies the demand\'s clarification. "\n        "Return at most 3 demands. "\n        "If nothing matches, return an empty demandIds array. "\n        "Probabilities must be in [0.0, 1.0]. "\n        "Return JSON only."\n    )\n\n    user = f"""Demands (id, name, clarification):\n{demands_context}\n\nChunkId: {chunk_id}\nChunk text:\n{chunk_text}\n\nReturn JSON in this format exactly:\n{{\n  "chunkId": "{chunk_id}",\n  "demandIds": [{{"id":"<one of provided ids>","probability":0.85}}],\n  "explanation": "brief reason"\n}}""".strip()\n\n    resp = client.chat.completions.create(\n        model=AZURE_CHAT_DEPLOYMENT,\n        messages=[{"role": "system", "content": system}, {"role": "user", "content": user}],\n    )\n    content = resp.choices[0].message.content or ""\n    parsed = _safe_json_loads(content)\n    return parsed or {"chunkId": chunk_id, "demandIds": [], "explanation": "LLM returned non-JSON."}\n\ndef run(raw_data: Any) -> Dict[str, Any]:\n    request = json.loads(raw_data) if isinstance(raw_data, str) else raw_data\n    if not isinstance(request, dict):\n        raise ValueError("Request must be a JSON object/dict.")\n    if "document" not in request or "num_preds" not in request:\n        raise ValueError("Invalid input: expected \'document\' and \'num_preds\'.")\n\n    document = request["document"]\n    num_pred = int(request["num_preds"])\n\n    by_id = document.get("contentDomain", {}).get("byId", {})\n    if not isinstance(by_id, dict):\n        raise ValueError("document.contentDomain.byId must be an object/dict.")\n\n    document_demand_predictions = set()\n\n    for chunk_id, content in by_id.items():\n        text = str(content.get("text", "") or "")[:MAX_TEXT_CHARS]\n\n        if client is None or DEMAND_EMB is None or not DEMANDS:\n            content.update({"relevantProba": 0.0, "cdLogregPredictions": [], "cdTransformerPredictions": []})\n            continue\n\n        candidates = _retrieve_candidates(text, TOP_K_RAG)\n        llm_out = _llm_classify_chunk(chunk_id, text, candidates)\n\n        preds = []\n        for item in (llm_out.get("demandIds", []) or [])[:MAX_LABELS_PER_CHUNK]:\n            did = str(item.get("id", "")).strip()\n            proba = _safe_float(item.get("probability", 0.0))\n            if did and proba >= MIN_KEEP_PROBA:\n                preds.append({"label": did, "proba": proba})\n                document_demand_predictions.add(did)\n\n        preds = sorted(preds, key=lambda x: x["proba"], reverse=True)[:num_pred]\n        relevant_proba = max([p["proba"] for p in preds], default=0.0)\n\n        content.update({"relevantProba": relevant_proba, "cdLogregPredictions": [], "cdTransformerPredictions": preds})\n\n    document["documentDemandPredictions"] = list(document_demand_predictions)\n    return {"predictions": document}\n', encoding="utf-8")
print("Wrote:", score_path, "bytes:", score_path.stat().st_size)

# Minimalny plik testowy request
sample_request = {
  "chunkId": "c1",
  "text": "The motor frame shall be connected to protective earth (PE). The enclosure shall be at least IP55."
}
(Path("./sample_request.json")).write_text(__import__("json").dumps(sample_request, indent=2), encoding="utf-8")
print("Wrote: sample_request.json")


## 5) Połączenie do Azure ML (SDK v2) + sanity check istniejącego endpointu

Jeżeli tu jest problem, to najczęściej:
- brak zalogowania (credential),
- zły subscription/RG/workspace,
- brak uprawnień.

W przypadku błędów wypiszemy konkret: error code + message.


In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError

credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)

ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WORKSPACE_NAME
)

try:
    endpoint = ml_client.online_endpoints.get(name=ENDPOINT_NAME)
    print("Endpoint FOUND:", endpoint.name)
    print("  auth_mode:", endpoint.auth_mode)
    print("  scoring_uri:", endpoint.scoring_uri)
    print("  traffic:", endpoint.traffic)
except HttpResponseError as e:
    print("FAILED to get endpoint. Details:")
    print("status:", getattr(e, "status_code", None))
    print("error:", e)
    raise


## 6) Lista deploymentów pod endpointem + podstawowa diagnostyka

Jeśli endpoint istnieje, ale deploymentów nie widać, zwykle oznacza:
- inny workspace niż myślisz,
- brak uprawnień do read na deployments.


In [ ]:
deployments = list(ml_client.online_deployments.list(endpoint_name=ENDPOINT_NAME))
print("Deployments under endpoint:", len(deployments))
for d in deployments:
    print("-", d.name, "| instance:", getattr(d, "instance_type", None), "| state:", getattr(d, "provisioning_state", None))


## 7) Utwórz/odśwież Environment asset z `conda_env.yml`

To pozwala wykryć błędy typu:
- brak paczki,
- konflikt wersji,
- zły format conda YAML.

Jeśli build środowiska się wywala, ten krok jest kluczowy.


In [ ]:
from azure.ai.ml.entities import Environment

env = Environment(
    name=ENV_NAME,
    version=ENV_VERSION,
    description="Env for Azure ML online deployment calling Azure OpenAI via openai SDK",
    conda_file=str(CONDA_YAML_PATH),
    image="mcr.microsoft.com/azureml/minimal-ubuntu22.04-py310-cpu-inference:latest"
)

try:
    env_result = ml_client.environments.create_or_update(env)
    print("Environment registered:", env_result.name, env_result.version)
except HttpResponseError as e:
    print("FAILED to create/update environment.")
    print("status:", getattr(e, "status_code", None))
    print("error:", e)
    raise


## 8) (Opcjonalnie) Usuń istniejący deployment o tej samej nazwie

To jest bezpieczna opcja na „zamrożone” deploymenty / konflikty.
Jeżeli nie chcesz usuwać, ustaw `DELETE_EXISTING=False`.


In [ ]:
from azure.core.exceptions import ResourceNotFoundError

DELETE_EXISTING = True

if DELETE_EXISTING:
    try:
        ml_client.online_deployments.get(name=NEW_DEPLOYMENT_NAME, endpoint_name=ENDPOINT_NAME)
        print("Deployment exists -> deleting:", NEW_DEPLOYMENT_NAME)
        poller = ml_client.online_deployments.begin_delete(name=NEW_DEPLOYMENT_NAME, endpoint_name=ENDPOINT_NAME)
        poller.result()
        print("Deleted:", NEW_DEPLOYMENT_NAME)
    except ResourceNotFoundError:
        print("Deployment not found -> nothing to delete.")
    except HttpResponseError as e:
        print("FAILED to delete deployment.")
        print("status:", getattr(e, "status_code", None))
        print("error:", e)
        raise
else:
    print("Skipping delete.")


## 9) Utwórz nowy Managed Online Deployment (ten, który ma dostać 100% traffic)

Uwaga: tutaj najczęściej „pada” na:
- build image (conda),
- brak dostępu do ACR,
- problem w `score.py` (np. importy),
- brak plików w `code_configuration`.

Po utworzeniu deploymentu od razu pobierzemy logi.


In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

deployment = ManagedOnlineDeployment(
    name=NEW_DEPLOYMENT_NAME,
    endpoint_name=ENDPOINT_NAME,
    environment=f"{ENV_NAME}:{ENV_VERSION}",
    code_configuration=CodeConfiguration(code=str(CODE_DIR), scoring_script="score.py"),
    instance_type=INSTANCE_TYPE,
    instance_count=INSTANCE_COUNT,
    # Wszelkie env vars do score.py (Azure OpenAI) zostawiamy do konfiguracji na deployment
    # Ustaw je w AML UI albo tutaj w dict. Bez tego init() w score.py się wywali.
    environment_variables={
        # Azure OpenAI
        "AZURE_OPENAI_ENDPOINT": os.getenv("AZURE_OPENAI_ENDPOINT",""),
        "AZURE_OPENAI_API_KEY": os.getenv("AZURE_OPENAI_API_KEY",""),
        "AZURE_OPENAI_API_VERSION": os.getenv("AZURE_OPENAI_API_VERSION","2024-02-01"),
        "AZURE_OPENAI_CHAT_DEPLOYMENT": os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT",""),
        "AZURE_OPENAI_EMBED_DEPLOYMENT": os.getenv("AZURE_OPENAI_EMBED_DEPLOYMENT",""),
        # scoring tweaks
        "TOP_K_RAG": os.getenv("TOP_K_RAG","6"),
        "MAX_LABELS_PER_CHUNK": os.getenv("MAX_LABELS_PER_CHUNK","3"),
        "MIN_KEEP_PROBA": os.getenv("MIN_KEEP_PROBA","0.25"),
        "MAX_TEXT_CHARS": os.getenv("MAX_TEXT_CHARS","8000"),
    }
)

try:
    poller = ml_client.online_deployments.begin_create_or_update(deployment)
    dep_result = poller.result()
    print("Deployment created:", dep_result.name, "| state:", dep_result.provisioning_state)
except HttpResponseError as e:
    print("FAILED to create/update deployment.")
    print("status:", getattr(e, "status_code", None))
    print("error:", e)
    # Spróbuj logi nawet jeśli create się wywalił (czasem już istnieją artefakty)
    raise


## 10) Logi deploymentu (jeśli coś nie działa, to jest pierwszy „go-to”)

Jeżeli init() rzuca wyjątek (np. brakuje env varów OpenAI), zobaczysz to tutaj.


In [ ]:
try:
    logs = ml_client.online_deployments.get_logs(
        name=NEW_DEPLOYMENT_NAME,
        endpoint_name=ENDPOINT_NAME,
        lines=200
    )
    print(logs)
except HttpResponseError as e:
    print("FAILED to get logs.")
    print("status:", getattr(e, "status_code", None))
    print("error:", e)


## 11) Smoke test — wywołanie endpointu

Wariant A: `ml_client.online_endpoints.invoke(...)` (jeśli dostępne w Twojej wersji SDK).
Wariant B: bezpośredni `requests.post()` na `scoring_uri` + key z `get_keys()`.

Jeśli smoke test nie przechodzi, **NIE przestawiamy trafficu**.


In [ ]:
import json, requests
from azure.core.exceptions import HttpResponseError

# pobierz key (działa, jeśli endpoint auth_mode=key)
try:
    keys = ml_client.online_endpoints.get_keys(name=ENDPOINT_NAME)
    primary_key = keys.primary_key
    print("Got endpoint key (primary):", "OK" if primary_key else "EMPTY")
except HttpResponseError as e:
    primary_key = None
    print("Could not get keys (maybe aml_token auth). Error:", e)

endpoint = ml_client.online_endpoints.get(name=ENDPOINT_NAME)
scoring_uri = endpoint.scoring_uri
headers = {"Content-Type": "application/json"}
if primary_key:
    headers["Authorization"] = f"Bearer {primary_key}"

payload = json.loads(Path("sample_request.json").read_text(encoding="utf-8"))

print("Calling scoring_uri:", scoring_uri)
resp = requests.post(scoring_uri, headers=headers, json=payload, timeout=120)
print("Status:", resp.status_code)
print(resp.text[:2000])
resp.raise_for_status()


## 12) Przestawienie trafficu — konfigurowalne (bez hardcode)

Zasada:
- Ustawiamy traffic dopiero po udanym smoke teście.
- Traffic to zwykły dict `{deployment_name: percent}`. citeturn0search3

Ważne: to aktualizuje **endpoint** (nie deployment).


In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

def set_traffic(traffic: dict):
    ep = ml_client.online_endpoints.get(name=ENDPOINT_NAME)
    ep.traffic = traffic
    poller = ml_client.online_endpoints.begin_create_or_update(ep)
    result = poller.result()
    return result

if SET_TRAFFIC_AFTER_SMOKE_TEST:
    traffic = {NEW_DEPLOYMENT_NAME: NEW_DEPLOYMENT_TRAFFIC_PERCENT}
    # Jeśli nie ustawiasz 100%, możesz dopisać pozostałe deploymenty ręcznie:
    # traffic = {NEW_DEPLOYMENT_NAME: 70, "old_blue": 30}
    updated = set_traffic(traffic)
    print("Updated traffic:", updated.traffic)
else:
    print("Skipping traffic update (SET_TRAFFIC_AFTER_SMOKE_TEST=False).")


## 13) Kontrola końcowa: endpoint traffic + lista deploymentów

Jeśli w UI masz „zamrożone” suwaki trafficu, zwykle znaczy, że UI blokuje edycję w pewnych stanach,
ale SDK/CLI nadal powinny pozwolić na update `endpoint.traffic`.


In [ ]:
endpoint = ml_client.online_endpoints.get(name=ENDPOINT_NAME)
print("Final traffic:", endpoint.traffic)

deployments = list(ml_client.online_deployments.list(endpoint_name=ENDPOINT_NAME))
print("Deployments:")
for d in deployments:
    print("-", d.name, "| state:", getattr(d, "provisioning_state", None))
